In [4]:
import os
import pandas as pd
import sys
import json
from google.protobuf.json_format import Parse

sys.path.insert(0, 'compiled_protobufs')
from taskmap_pb2 import TaskMap

qrels_folder = os.path.join(os.getcwd(), 'measurements', 'judgments')
run_cooking_folder = os.path.join(os.getcwd(), 'measurements', 'cooking', 'run_files')
run_diy_folder = os.path.join(os.getcwd(), 'measurements', 'diy', 'run_files')


In [34]:
# ## save qrels files

# with open(qrels_path, "w") as f:
#     pd_annotations = pd.read_csv(annotations_path)
#     lines = []
#     # print(pd_annotations.head())
#     for idx, annotation in pd_annotations.iterrows():
#         q_id, doc_id, score = annotation["query-id"], annotation["doc-id"], annotation["relevance"]
#         lines.append(f'{q_id} Q0 {doc_id} {score}\n')
#     lines[-1] = lines[-1].replace("\n","")
#     f.writelines(lines)
    

In [54]:
# run_path = os.path.join(os.getcwd(), 'datasets', 'qrles')
# taskmap_cooking_index_path = os.path.join(os.getcwd(), "indexes", "food", "system_index_sparse")


In [69]:

# from pygaggle.rerank.base import Query, Text, hits_to_texts
# from pygaggle.rerank.transformer import MonoT5

# config = [
#     "bm25",
#     "bm25+rm3",
#     "bm25+t5",
#     "bm25+rm3+t5",
# ]

In [1]:


# cooking_queries = pd.read_csv(queries_path).iloc[:10]
# reranker =  MonoT5()

# def get_searcher(search_model):
#     if search_model == "bm25" or search_model == "bm25+t5":
#         searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
#         searcher.set_bm25(b=0.4, k1=0.9)
#     if search_model == "bm25+rm3" or search_model == "bm25+rm3+t5":
#         searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
#         searcher.set_bm25(b=0.4, k1=0.9)
#         searcher.set_rm3(fb_terms=10, fb_docs=10, original_query_weight=0.5)
#     return searcher

# for model in config:
#     lines = []
#     for idx, query in cooking_queries.iterrows():
#         print(f"Initialize searcher {model}")
#         searcher = get_searcher(model)
#         hits = searcher.search(q=query["target query"], k=50)
#         if "t5" in model:
#             hits = reranker.rerank(Query(query["target query"]), hits_to_texts(hits))
#         for rank, hit in enumerate(hits):
#             if type(hit) == Text:
#                 doc_json = json.loads(hit.text)
#             else:
#                 doc_json = json.loads(hit.raw)
#             taskmap_json = doc_json['recipe_document_json']
#             taskmap = Parse(json.dumps(taskmap_json), TaskMap())
#             doc_id = taskmap.taskmap_id
#             line = f'query-{idx} Q0 {doc_id} {rank+1} {hit.score} bm25\n'
#             lines.append(line)
#     lines[-1] = lines[-1].replace("\n","")

#     with open(os.path.join(run_path, model+".run"), "w") as f:
#         f.writelines(lines)
        



In [22]:
import ir_measures
from ir_measures import *

# qrles = ir_measures.read_trec_qrels('qrels/qrls.qrles')
# run = ir_measures.read_trec_run('qrels.run')

# for model in config:
#     print(os.path.join(run_path, model+".run"))
#     qrles = ir_measures.read_trec_qrels(qrels_path)
#     run = ir_measures.read_trec_run(os.path.join(run_path, model+".run"))
#     accuracy = ir_measures.calc_aggregate([nDCG@3, Precision@3, Recall@3], qrles, run)
#     print(f"{model}: {accuracy}")

def get_accuracy(run_filepath, qrles_filepath):
    run = ir_measures.read_trec_run(run_filepath)
    qrles = ir_measures.read_trec_qrels(qrles_filepath)
    accuracy = ir_measures.calc_aggregate([nDCG@10, Precision@3, Precision@5, Precision@10, MAP], qrles, run)
    return accuracy
    
#     accuracy = ir_measures.calc_aggregate([nDCG@3, Precision@3, Recall@3], qrles, run)

In [18]:
cooking_runs = os.listdir(run_cooking_folder)
cooking_qrles = os.path.join(qrels_folder, "cooking.qrles")

for searcher in cooking_runs:
    run = os.path.join(run_cooking_folder, searcher)
    searcher_name = searcher[:-4].lower()
    # print(searcher_name, run)
    accuracy = get_accuracy(run, cooking_qrles)
    print(searcher_name, accuracy)
    

bm25+rm3 {nDCG@3: nan, P@3: nan, R@3: nan}
marqo {nDCG@3: nan, P@3: nan, R@3: nan}
marqo-target {nDCG@3: nan, P@3: nan, R@3: nan}
bm25+rm3+t5 {nDCG@3: nan, P@3: nan, R@3: nan}
ance {nDCG@3: nan, P@3: nan, R@3: nan}
bm25 {nDCG@3: nan, P@3: nan, R@3: nan}
bm25+t5 {nDCG@3: nan, P@3: nan, R@3: nan}
tct-colbertv2 {nDCG@3: nan, P@3: nan, R@3: nan}


In [24]:
diy_runs = os.listdir(run_diy_folder)
diy_qrles = os.path.join(qrels_folder, "diy.qrles")

results = {}
for searcher in diy_runs:
    run = os.path.join(run_diy_folder, searcher)
    searcher_name = searcher[:-4].lower()
    # print(searcher_name, run)
    accuracy = get_accuracy(run, diy_qrles)
    results[searcher_name] = accuracy
    print(searcher_name, accuracy)
    print(type(accuracy))
    

bm25+rm3 {P@5: 0.7019999999999998, AP: 0.5433643060232928, P@3: 0.7433333333333332, P@10: 0.6169999999999998, nDCG@10: 0.6170839613009708}
<class 'dict'>
marqo {P@5: 0.6599999999999998, AP: 0.4710119913660888, P@3: 0.7099999999999999, P@10: 0.5479999999999997, nDCG@10: 0.5797733430156824}
<class 'dict'>
marqo-target {P@5: 0.4340000000000001, AP: 0.3173515739176255, P@3: 0.44666666666666655, P@10: 0.3850000000000001, nDCG@10: 0.3858630486447604}
<class 'dict'>
bm25+rm3+t5 {P@5: 0.7819999999999996, AP: 0.6134608423061272, P@3: 0.8100000000000003, P@10: 0.7010000000000001, nDCG@10: 0.7214809440436319}
<class 'dict'>
ance {P@5: 0.5559999999999998, AP: 0.3333787621853429, P@3: 0.6499999999999997, P@10: 0.45500000000000007, nDCG@10: 0.5005271397466523}
<class 'dict'>
bm25 {P@5: 0.6919999999999998, AP: 0.5214069760454759, P@3: 0.7299999999999995, P@10: 0.5860000000000002, nDCG@10: 0.5968774759791069}
<class 'dict'>
bm25+t5 {P@5: 0.7679999999999997, AP: 0.5984733186917751, P@3: 0.79, P@10: 0.6